In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import pickle_data

import tf_lab.point_clouds.in_out as pio

from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch


from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.data_sets.shape_net import pc_loader as sn_pc_loader
from tf_lab.nips.helper import center_pclouds_in_unit_sphere
from tf_lab.icml.ae_farm_helper import EMD_Stats, Chamfer_Stats

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
from bench_pc_gan.evaluate_gan import entropy_of_occupancy_grid, jensen_shannon_divergence
from tf_lab.nips.helper import pclouds_centered_and_half_sphere
from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset
from general_tools.simpletons import iterate_in_chunks

PyTorch not working. MMD measurement won't be available


In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
class_name = 'chair'
ae_loss = 'emd'
n_pc_samples = 2048
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'

In [6]:
# Load Raw Point-Clouds of class
pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
syn_id = shape_net_category_to_synth_id()[class_name]
pclouds_path = osp.join(pclouds_path, syn_id)
file_names = pio.load_filenames_of_input_data(pclouds_path, '.ply')
pclouds, model_ids, syn_ids = pio.load_crude_point_clouds(file_names=file_names, n_threads=50, loader=sn_pc_loader)
print '%d files containing complete point clouds were found.' % (len(pclouds), )
train_data = PointCloudDataSet(pclouds)

6778 files containing complete point clouds were found.


In [7]:
cmp_in_sphere = True
voxel_resolution = 28
pclouds_ = pclouds_centered_and_half_sphere(pclouds)
_, gt_grid_var = entropy_of_occupancy_grid(pclouds_, voxel_resolution, in_sphere=cmp_in_sphere)

In [18]:
ae_id = 16
ae_name = '_'.join(['ae_farm', class_name, 'conv_arch', str(ae_id), str(n_pc_samples) + 'pts', ae_loss])
train_dir = osp.join(top_data_dir, 'OUT/icml/nn_models/ae_farming', ae_name)
conf = Conf.load(osp.join(train_dir, 'configuration'))
reset_tf_graph()
ae = PointNetAutoEncoder(ae_name, conf)    
print ae_name
load_epoch = 1000
ae.restore_model(conf.train_dir, load_epoch, verbose=True)
_, latent_codes, _ = latent_embedding_of_entire_dataset(train_data, ae, conf)
reconstructed = []
for b in iterate_in_chunks(latent_codes, conf.batch_size):
    reconstructed.append(ae.decode(b))
reconstructed = np.vstack(reconstructed) 
reconstructed = pclouds_centered_and_half_sphere(reconstructed)

from sklearn import mixture

for n_component in [10]:
    print n_component
    gmm = mixture.GaussianMixture(n_component, covariance_type='full')
    gmm.fit(latent_codes)
#     sample_codes = gmm.sample(raw_data.num_examples)[0]
    print gmm.bic(latent_codes)    
    print gmm.score(latent_codes)

ae_farm_chair_conv_arch_16_2048pts_emd
Model restored in epoch 1000.
10
42278.0653729
-2.12378929341


In [11]:
experiment_stats = EMD_Stats
ae_ids = experiment_stats.experiments_ids
exp_type = 'best'

ae_ids = [18]

for ae_id in ae_ids:
    ae_name = '_'.join(['ae_farm', class_name, 'conv_arch', str(ae_id), str(n_pc_samples) + 'pts', ae_loss])
    train_dir = osp.join(top_data_dir, 'OUT/icml/nn_models/ae_farming', ae_name)
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    bneck = conf.encoder_args['n_filters'][-1]
    assert(experiment_stats.experiment_id_to_bneck[ae_id] == bneck)
    print bneck
    
    if exp_type == 'max_min':
        load_epoch = experiment_stats.max_min_epochs[bneck][0]
    else:
        load_epoch = experiment_stats.best_epochs[bneck][0]
    
    reset_tf_graph()
    ae = PointNetAutoEncoder(ae_name, conf)    
    print ae_name
    ae.restore_model(conf.train_dir, load_epoch, verbose=True)
    _, latent_codes, _ = latent_embedding_of_entire_dataset(train_data, ae, conf)
    reconstructed = []
    for b in iterate_in_chunks(latent_codes, conf.batch_size):
        reconstructed.append(ae.decode(b))
    reconstructed = np.vstack(reconstructed) 
    reconstructed = pclouds_centered_and_half_sphere(reconstructed)
    _, sample_grid_var = entropy_of_occupancy_grid(reconstructed, voxel_resolution, in_sphere=cmp_in_sphere)
    jsd_epoch = jensen_shannon_divergence(sample_grid_var, gt_grid_var)
    print jsd_epoch

[10, 11, 12, 13, 14, 15]
4
ae_farm_chair_conv_arch_18_2048pts_emd
Model restored in epoch 1000.
0.0030742091352


In [15]:
# Best_Epoch
(4, 0.0030742091352)
(8, 0.00135260351788)
(16, 0.000822138606231)
(32, 0.000644479942822)
(64, 0.000647472505707)
(128, 0.000561809067982)
(256, 0.000567715030499)
(512, 0.000554273937081)
(1024, 0.000541790000585)

# Max_min Epoch
(32, 0.000644479942822)
(64, 0.000733603530406)
(128, 0.000684516282747)
(256, 0.000773909275057)
(512, 0.000771209492351)